In [ ]:
! pip install -U git+https://github.com/IINemo/isanlp.git@discourse

In [1]:
from isanlp import PipelineCommon
from isanlp.processor_remote import ProcessorRemote
from isanlp.ru.processor_mystem import ProcessorMystem
from isanlp.ru.converter_mystem_to_ud import ConverterMystemToUd

SERVER = ''  # put the address here
address_syntax = (SERVER, 3344)
address_rst = (SERVER, 3335)

ppl = PipelineCommon([
    (ProcessorRemote(address_syntax[0], address_syntax[1], '0'),
     ['text'],
     {'sentences': 'sentences',
      'tokens': 'tokens',
      'lemma': 'lemma',
      'syntax_dep_tree': 'syntax_dep_tree',
      'postag': 'ud_postag'}),
    (ProcessorMystem(delay_init=False),
     ['tokens', 'sentences'],
     {'postag': 'postag'}),
    (ConverterMystemToUd(),
     ['postag'],
     {'morph': 'morph',
      'postag': 'postag'}),
    (ProcessorRemote(address_rst[0], address_rst[1], 'default'),
     ['text', 'tokens', 'sentences', 'postag', 'morph', 'lemma', 'syntax_dep_tree'],
     {'rst': 'rst'})
])

In [2]:
text = (
"Новости о грядущей эмиссии в США обвалили доллар и подняли цену золота. При этом рост количества "
"долларов пока не зафиксирован. Со швейцарским франком ситуация противоположная. Стало известно, ч"
"то в феврале денежная масса Швейцарии увеличилась на 3.5%, однако биржевой курс франка и его покуп"
"ательная способность за неделю выросли."
)

In [3]:
%%time

result = ppl(text)

CPU times: user 20.9 ms, sys: 785 µs, total: 21.7 ms
Wall time: 6.04 s


In [4]:
result['rst']

In [5]:
vars(result['rst'][0])

{'id': 10,
 'left': <isanlp.annotation_rst.DiscourseUnit at 0x7ff641d29e10>,
 'right': <isanlp.annotation_rst.DiscourseUnit at 0x7ff641d2a110>,
 'relation': 'interpretation-evaluation',
 'nuclearity': 'NS',
 'proba': 0.26196339444349426,
 'start': 0,
 'end': 331,
 'text': 'Новости о грядущей эмиссии в США обвалили доллар и подняли цену золота. При этом рост количества долларов пока не зафиксирован. Со швейцарским франком ситуация противоположная. Стало известно, что в феврале денежная масса Швейцарии увеличилась на 3.5%, однако биржевой курс франка и его покупательная способность за неделю выросли.'}

In [6]:
def extr_pairs(tree, text):
    pp = []
    if tree.left:
        pp.append([text[tree.left.start:tree.left.end], text[tree.right.start:tree.right.end], tree.relation, tree.nuclearity])
        pp += extr_pairs(tree.left, text)
        pp += extr_pairs(tree.right, text)
    return pp

In [7]:
extr_pairs(result['rst'][0], result['text'])

[['Новости о грядущей эмиссии в США обвалили доллар и подняли цену золота. При этом рост количества долларов пока не зафиксирован.',
  'Со швейцарским франком ситуация противоположная. Стало известно, что в феврале денежная масса Швейцарии увеличилась на 3.5%, однако биржевой курс франка и его покупательная способность за неделю выросли.',
  'interpretation-evaluation',
  'NS'],
 ['Новости о грядущей эмиссии в США обвалили доллар и подняли цену золота.',
  'При этом рост количества долларов пока не зафиксирован.',
  'elaboration',
  'NS'],
 ['Новости о грядущей эмиссии в США обвалили доллар',
  'и подняли цену золота.',
  'joint',
  'NN'],
 ['Со швейцарским франком ситуация противоположная.',
  'Стало известно, что в феврале денежная масса Швейцарии увеличилась на 3.5%, однако биржевой курс франка и его покупательная способность за неделю выросли.',
  'interpretation-evaluation',
  'NS'],
 ['Стало известно, что в феврале денежная масса Швейцарии увеличилась на 3.5%,',
  'однако биржево

In [8]:
import sys
sys.path.append('../')
sys.path.append('../../')

from _isanlp_rst.src.isanlp_rst.export.to_rs3 import ForestExporter

exporter = ForestExporter(encoding='utf8')
exporter(result['rst'], 'example.rs3')

Output:

![example.rs3](example.rs3.png)